# Preparation for Facebook Technical Questions

Here are my notes for preparing for the technical portion of the interview. To simulate interview conditions, I wrote all code in markdown mode only converting the cells to executable after I felt comfortable they were correct.

[Exercises](#exercises)

[Notes](#notes)

## Exercises <a id='exercises'></a>

### Mock Question from E-mail

An attendance log for every student in a school district ```attendance_events```:

| date | student_id | attendance |
|:----:|:----------:|:----------:|
|      |            |            |

A summary table with demographics for each student in the district ```all_students```: 

|student_id | school_id | grade_level | date_of_birth | hometown |
|-----------|-----------|-------------|---------------|----------|

Using this data, you could answer questions like the following:

* What percent of students attend school on their birthday?
* Which grade level had the largest drop in attendance between yesterday and today?

In [214]:
################################################################################
################################################################################
# Functions used to generate mock data simulating the tables given above.      #
################################################################################
################################################################################

import pandas as pd
import numpy as np

n_students = 1000
n_days = 10
start_date = '2017-09-01'
end_date = '2018-06-15'

################################################################################
# Attendance Table                                                             #
################################################################################
def _make_attendance_dates(n_students, start_date, end_date):
    dr = pd.date_range(start_date, end_date)
    dates = []
    for day in dr:
        dates.extend([day] * n_students)
    return dates

def _make_student_ids(n_students, n_days=None):
    student_ids = [xx for xx in range(100, 100 + n_students)]
    if n_days is not None:
        student_ids = student_ids * n_days
    return student_ids

def _make_attendance(n_students, n_days):
    attendance = []
    for _ in range(0, n_days):
        prob = np.random.rand()
        this_attendance = list(np.random.choice(2, n_students, p=[prob, 1.-prob]))
        attendance.extend(this_attendance)
    return attendance

def build_attendance(n_students, start_date, end_date):
    columns = ['date', 'student_id', 'attendance']
    n_days = len(pd.date_range(start_date, end_date))
    
    dates       = _make_attendance_dates(n_students, start_date, end_date)
    student_ids = _make_student_ids(n_students, n_days)
    attendance  = _make_attendance(n_students, n_days)
    data = [xx for xx in zip(dates, student_ids, attendance)]
    
    df = pd.DataFrame(data=data, columns=columns)
    return df

################################################################################
# District All Students Table                                                  #
################################################################################
def _make_school_ids(n_students):
    schools = ['South River High School',
               'New Brunswick High School',
               'East Brunswick High School',
               'Edison High School']
    return list(np.random.choice(schools, n_students))

def _make_grade_levels(n_students):
    grades = ['Freshman', 'Sophomore', 'Junior', 'Senior']
    return list(np.random.choice(grades, n_students))
    
def _make_DOBs(grade_levels):
    birth_years = {
        'Freshman': 2005,
        'Sophomore': 2004,
        'Junior': 2003,
        'Senior': 2002
    }
    years = [birth_years[xx] for xx in grade_levels]
    months = list(np.random.choice(np.arange(1,13), len(grade_levels)))
    days = list(np.random.choice(np.arange(1,29), len(grade_levels)))
    DOBs = pd.to_datetime(['{}-{}-{}'.format(*dd) for dd in zip(months, days, years)])
    return DOBs

def _make_hometowns(school_ids):
    hometowns = [school.split('High School')[0].strip() for school in school_ids]
    return hometowns
    

def build_all_students(n_students):
    student_ids  = _make_student_ids(n_students)
    school_ids   = _make_school_ids(n_students)
    grade_levels = _make_grade_levels(n_students)
    DOBs         = _make_DOBs(grade_levels)
    hometowns    = _make_hometowns(school_ids)
    
    columns = ['student_id', 'school_id', 'grade_level', 'date_of_birth', 'hometown']
    data = [xx for xx in zip(student_ids, school_ids, grade_levels, DOBs, hometowns)]
    df = pd.DataFrame(data=data, columns=columns)
    return df
    

attendance = build_attendance(n_students, start_date, end_date)
all_students = build_all_students(n_students=n_students)

In [216]:
attendance.tail()

,date,student_id,attendance
287995,2018-06-15,1095,1
287996,2018-06-15,1096,1
287997,2018-06-15,1097,1
287998,2018-06-15,1098,1
287999,2018-06-15,1099,0


In [217]:
all_students.head()

,student_id,school_id,grade_level,date_of_birth,hometown
0,100,South River High School,Sophomore,2004-07-12,South River
1,101,East Brunswick High School,Freshman,2005-09-09,East Brunswick
2,102,East Brunswick High School,Junior,2003-05-19,East Brunswick
3,103,Edison High School,Senior,2002-06-28,Edison
4,104,South River High School,Junior,2003-07-05,South River


#### What percent of students attend school on their birthday?

In [218]:
all_attendance = pd.merge(
    left=attendance,
    right=all_students,
    how='inner',
    on='student_id'
)

attended = (all_attendance['attendance'] == 1)

attendance_month = all_attendance['date'].apply(lambda x: x.month)
attendance_day = all_attendance['date'].apply(lambda x: x.day)
birthday_month = all_attendance['date_of_birth'].apply(lambda x: x.month)
birthday_day = all_attendance['date_of_birth'].apply(lambda x: x.day)

on_birthday = (attendance_month == birthday_month) & (attendance_day == birthday_day)

attended_on_birthday = all_attendance[attended & on_birthday]

num_attended_on_birthday = float(attended_on_birthday.shape[0])
num_total = float(all_students.shape[0])

pct_attended_on_birthday = (num_attended_on_birthday / num_total) * 100.
pct_attended_on_birthday

42.1

**Which grade level had the largest drop in attendance between yesterday and today?**

In [200]:
date_attendance = all_attendance[all_attendance['date'] == today]

In [236]:
today = pd.to_datetime('2017-09-04')
yesterday = pd.to_datetime('2017-09-03')

def get_attendance(date):
    date_attendance = all_attendance[all_attendance['date'] == date]
    date_attendance = date_attendance[['grade_level', 'attendance']]
    date_attendance = date_attendance[date_attendance['attendance'] == 1]
    num_attended = date_attendance.groupby('grade_level').sum()
    return num_attended

today_attendance = get_attendance(today)
yesterday_attendance = get_attendance(yesterday)

drop_attendance = today_attendance - yesterday_attendance
drop_attendance.idxmax()['attendance']

'Sophomore'

## LeetCode

### Second/N-th Highest Salary

<div class="question-description__3U1T"><div><p>Write a SQL query to get the second highest salary from the <code>Employee</code> table.</p>

<pre>+----+--------+
| Id | Salary |
+----+--------+
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |
+----+--------+
</pre>

<p>For example, given the above Employee table, the query should return <code>200</code> as the second highest salary. If there is no second highest salary, then the query should return <code>null</code>.</p>

<pre>+---------------------+
| SecondHighestSalary |
+---------------------+
| 200                 |
+---------------------+
</pre>
</div></div>

In [248]:
import pandas as pd
import numpy as np

num_employees = 10

def build_employee_table(num_employees):
    salaries = np.random.choice(np.arange(100, 1000, 100), 10)
    Employee = pd.DataFrame(salaries, columns=['salary'])
    return Employee

Employee = build_employee_table(num_employees)

def get_nth_highest_salary(employee_table, n):
    # Note: 
    #   * can use pd.unique() or Series.unique()
    #   * for pd.unique() can pass it any array like object: Series, np.array, list
    salaries = employee_table['salary'].unique()
    
    # Note: np.sort() returns a sorted copy
    #   ndarray.sort() sorts in place
    #   can't specify ascending/descneding
    salaries_sort = np.sort(salaries)
    return salaries_sort, salaries_sort[-n]
    
print(Employee)
print(get_nth_highest_salary(Employee, 2))

   salary
0     300
1     300
2     100
3     200
4     400
5     600
6     800
7     900
8     900
9     400
(array([100, 200, 300, 400, 600, 800, 900]), 800)


### Customers Who Never Order

<div class="question-detail"><div class="question-description__3U1T"><div><p>Suppose that a website contains two tables, the <code>Customers</code> table and the <code>Orders</code> table. Write a SQL query to find all customers who never order anything.</p>

<p>Table: <code>Customers</code>.</p>

<pre>+----+-------+
| Id | Name  |
+----+-------+
| 1  | Joe   |
| 2  | Henry |
| 3  | Sam   |
| 4  | Max   |
+----+-------+
</pre>

<p>Table: <code>Orders</code>.</p>

<pre>+----+------------+
| Id | CustomerId |
+----+------------+
| 1  | 3          |
| 2  | 1          |
+----+------------+
</pre>

<p>Using the above tables as example, return the following:</p>

<pre>+-----------+
| Customers |
+-----------+
| Henry     |
| Max       |
+-----------+
</pre>
</div></div></div>

In [267]:
import pandas as pd
import numpy as np

# Notes:
#   * Set index name after construction
Customers = pd.DataFrame(
    data=['Joe', 'Henry', 'Sam', 'Max'],
    index=np.arange(1,5),
    columns=['Name']
)
Customers.index.name = 'Id'

Orders = pd.DataFrame(
    data=[3, 1],
    index=np.arange(1,3),
    columns=['CustomerId']
)

Customer_Orders = pd.merge(
    left=Orders,
    right=Customers,
    how='outer',
    left_on=['CustomerId'],
    right_on=['Id']
)

# To get nan's use pd.isnull()
never_ordered = Customer_Orders[pd.isnull(Customer_Orders['CustomerId'])]['Name']
never_ordered = Customer_Orders[Customer_Orders['CustomerId'].isna()]['Name']
never_ordered

2    Henry
3      Max
Name: Name, dtype: object

### Friend Requests I: Overall Acceptance Rate

<div class="question-area"><div class="question-detail"><div class="question-description__3U1T"><div>In social network like Facebook or Twitter, people send friend requests and accept others’ requests as well. Now given two tables as below:<p></p>

Table: <code>friend_request</code>
<pre>| sender_id | send_to_id |request_date|
|-----------|------------|------------|
| 1         | 2          | 2016_06-01 |
| 1         | 3          | 2016_06-01 |
| 1         | 4          | 2016_06-01 |
| 2         | 3          | 2016_06-02 |
| 3         | 4          | 2016-06-09 |
</pre><p></p>

Table: <code>request_accepted</code>
<pre>| requester_id | accepter_id |accept_date |
|--------------|-------------|------------|
| 1            | 2           | 2016_06-03 |
| 1            | 3           | 2016-06-08 |
| 2            | 3           | 2016-06-08 |
| 3            | 4           | 2016-06-09 |
| 3            | 4           | 2016-06-10 |
</pre><p></p>

Write a query to find the overall acceptance rate of requests rounded to 2 decimals, which is the number of acceptance divide the number of requests.<p></p>

For the sample data above, your query should return the following result.<p></p>
<pre>|accept_rate|
|-----------|
|       0.80|
</pre><p></p>

<b>Note:</b>
<li>The accepted requests are not necessarily from the table <code>friend_request</code>. In this case, you just need to simply count the total accepted requests (no matter whether they are in the original requests), and divide it by the number of requests to get the acceptance rate.</li>
<li>It is possible that a sender sends multiple requests to the same receiver, and a request could be accepted more than once. In this case, the ‘duplicated’ requests or acceptances are only counted once.</li>
<li>If there is no requests at all, you should return 0.00 as the accept_rate. </li>
<p></p>

<b>Explanation:</b> There are 4 unique accepted requests, and there are 5 requests in total. So the rate is 0.80.<p></p>

<b>Follow-up:</b><br>
<li>Can you write a query to return the accept rate but for every month?</li>
<li>How about the cumulative accept rate for every day?</li></div></div></div></div>

In [298]:
def build_friend_request():
    friend_request = {
        'sender_id':  [1, 1, 1, 2, 3],
        'send_to_id': [2, 3, 4, 3, 4],
        'request_date': pd.to_datetime([
            '2016-06-01',
            '2016-06-01',
            '2016-06-01',
            '2016-06-02',
            '2016-06-09'
        ])
    }
    return pd.DataFrame(friend_request)
    
def build_request_accepted():
    request_accepted = {
        'requester_id':  [1, 1, 2, 3, 3],
        'accepter_id': [2, 3, 3, 4, 4],
        'accept_date': pd.to_datetime([
            '2016-06-03',
            '2016-06-08',
            '2016-06-08',
            '2016-06-09',
            '2016-06-10'
        ])
    }
    return pd.DataFrame(request_accepted)
    
friend_request = build_friend_request()
friend_request_unique = friend_request.drop_duplicates(subset=['sender_id', 'send_to_id'])

request_accepted = build_request_accepted()
request_accepted_unique = request_accepted.drop_duplicates(subset=['requester_id', 'accepter_id'])

acceptance_rate = float(request_accepted_unique.shape[0]) / friend_request_unique.shape[0]

acceptance_rate    

0.8

In [360]:
# Follow-Ups

def build_friend_request():
    friend_request = {
        'sender_id':  [1, 1, 1, 2, 3, 5, 6, 5],
        'send_to_id': [2, 3, 4, 3, 4, 4, 8, 6],
        'request_date': pd.to_datetime([
            '2016-06-01',
            '2016-06-01',
            '2016-06-01',
            '2016-06-02',
            '2016-06-09',
            '2016-07-02',
            '2016-07-03',
            '2016-07-10'
        ])
    }
    return pd.DataFrame(friend_request)
    
def build_request_accepted():
    request_accepted = {
        'requester_id':  [1, 1, 2, 3, 3, 8, 7, 6],
        'accepter_id':   [2, 3, 3, 4, 4, 9, 5, 5],
        'accept_date': pd.to_datetime([
            '2016-06-03',
            '2016-06-08',
            '2016-06-08',
            '2016-06-09',
            '2016-06-10',
            '2016-07-10',
            '2016-07-10',
            '2016-08-10'
        ])
    }
    return pd.DataFrame(request_accepted)
    
fr = build_friend_request()
fr_unique = fr.drop_duplicates(subset=['sender_id', 'send_to_id'])

ra = build_request_accepted()
ra_unique = ra.drop_duplicates(subset=['requester_id', 'accepter_id'])

fr_unique.index = fr_unique.pop('request_date')
ra_unique.index = ra_unique.pop('accept_date')

# Group By Month & Day

month_key = lambda x: x.month
day_key = lambda x: x.day

# Month
fr_month_counts = fr_unique.groupby(by=[month_key]).count().pop('sender_id')
fr_month_counts.name = 'fr_month_counts'
fr_month_counts = pd.DataFrame(fr_month_counts.apply(float))

ra_month_counts = ra_unique.groupby(by=[month_key]).count().pop('accepter_id')
ra_month_counts.name = 'ra_month_counts'
ra_month_counts = pd.DataFrame(ra_month_counts.apply(float))

accept_rate_month = pd.merge(
    left=ra_month_counts,
    right=fr_month_counts,
    how='outer',
    left_index=True,
    right_index=True
)

accept_rate_month['acceptance_rate'] = accept_rate_month.apply(lambda x: x['ra_month_counts'] / x['fr_month_counts'], axis=1)
accept_rate_month = accept_rate_month.fillna(0.0)

# Day
fr_month_day_counts = fr_unique.groupby(by=[month_key, day_key]).count().pop('sender_id')
fr_month_day_counts.name = 'fr_month_day_counts'
fr_month_day_counts = pd.DataFrame(fr_month_day_counts.apply(float))

ra_month_day_counts = ra_unique.groupby(by=[month_key, day_key]).count().pop('accepter_id')
ra_month_day_counts.name = 'ra_month_day_counts'
ra_month_day_counts = pd.DataFrame(ra_month_day_counts.apply(float))
print(ra_month_day_counts)
print(fr_month_day_counts)

accept_rate_month_day = pd.merge(
    left=ra_month_day_counts,
    right=fr_month_day_counts,
    how='outer',
    left_index=True,
    right_index=True
)


ar_md_dict = {}
months = []
days = []
ar_list = []
for row in accept_rate_month_day.iterrows():
    this_month = row[0][0]
    this_day = row[0][1]
    this_fr = row[1].fr_month_day_counts
    this_ra = row[1].ra_month_day_counts
    this_accept_rate = this_ra / this_fr
    months.append(this_month)
    days.append(this_day)
    ar_list.append(this_accept_rate)
    
ar_dict = {
    'month': months,
    'day': days,
    'accept_rate': ar_list
}

accept_rate_month_day = pd.DataFrame(ar_dict).fillna(0)
accept_rate_month_day
    
    

      ra_month_day_counts
6 3                   1.0
  8                   2.0
  9                   1.0
7 10                  2.0
8 10                  1.0
      fr_month_day_counts
6 1                   3.0
  2                   1.0
  9                   1.0
7 2                   1.0
  3                   1.0
  10                  1.0


,accept_rate,day,month
0,0.0,1,6
1,0.0,2,6
2,0.0,3,6
3,0.0,8,6
4,1.0,9,6
5,0.0,2,7
6,0.0,3,7
7,2.0,10,7
8,0.0,10,8


In [380]:
# Should have done it this way
accept_rate_month_day = pd.merge(
    left=ra_month_day_counts,
    right=fr_month_day_counts,
    how='outer',
    left_index=True,
    right_index=True
)

def get_md_arr(idx):
    s = accept_rate_month_day.loc[idx]
    return s.ra_month_day_counts / s.fr_month_day_counts
accept_rate_month_day.apply(lambda x: get_md_arr(x.name), axis=1).fillna(0.0)



6  1     0.0
   2     0.0
   3     0.0
   8     0.0
   9     1.0
7  2     0.0
   3     0.0
   10    2.0
8  10    0.0
dtype: float64

#### Relevant Notes

[Pandas - Dates and Times](#pandas-dates)

[Pandas - Merge, Join, Concatenate](#pandas-join)

## Notes <a id='notes'></a>

<a id='pandas-dates'></a> [**Pandas - Time Series / Date Functionality**](http://pandas.pydata.org/pandas-docs/version/0.23/timeseries.html)

* Uses the numpy ```datetime64``` and ```timedelta64``` datatypes

<table border="1" class="docutils">
<colgroup>
<col width="15%">
<col width="27%">
<col width="58%">
</colgroup>

<thead valign="bottom">
<tr class="row-odd"><th class="head">Class</th>
<th class="head">Remarks</th>
<th class="head">How to create</th>
</tr>
</thead><tbody valign="top">
<tr class="row-even"><td><code class="docutils literal notranslate"><span class="pre">Timestamp</span></code></td>
<td>Represents a single timestamp</td>
<td><code class="docutils literal notranslate"><span class="pre">to_datetime</span></code>, <code class="docutils literal notranslate"><span class="pre">Timestamp</span></code></td>
</tr>
<tr class="row-odd"><td><code class="docutils literal notranslate"><span class="pre">DatetimeIndex</span></code></td>
<td>Index of <code class="docutils literal notranslate"><span class="pre">Timestamp</span></code></td>
<td><code class="docutils literal notranslate"><span class="pre">to_datetime</span></code>, <code class="docutils literal notranslate"><span class="pre">date_range</span></code>, <code class="docutils literal notranslate"><span class="pre">bdate_range</span></code>, <code class="docutils literal notranslate"><span class="pre">DatetimeIndex</span></code></td>
</tr>
<tr class="row-even"><td><code class="docutils literal notranslate"><span class="pre">Period</span></code></td>
<td>Represents a single time span</td>
<td><code class="docutils literal notranslate"><span class="pre">Period</span></code></td>
</tr>
<tr class="row-odd"><td><code class="docutils literal notranslate"><span class="pre">PeriodIndex</span></code></td>
<td>Index of <code class="docutils literal notranslate"><span class="pre">Period</span></code></td>
<td><code class="docutils literal notranslate"><span class="pre">period_range</span></code>, <code class="docutils literal notranslate"><span class="pre">PeriodIndex</span></code></td>
</tr>
</tbody>
</table>

* Both ```Timestamp``` and  ```Period``` objects can serve as an index. They are automatically cast into ```DatetimeIndex``` and ```PeriodIndex``` objects.

In [1]:
import pandas as pd
# Can convert from strings to date-like objects via pd.to_datetime
pd.to_datetime(pd.Series(['Jul 31, 2009', '2010-01-10', None]))

/home/casey/coding_interview/interview_venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


0   2009-07-31
1   2010-01-10
2          NaT
dtype: datetime64[ns]

In [2]:
# Make range of dates:
pd.date_range('2010-02-20', '2011-03-05')

DatetimeIndex(['2010-02-20', '2010-02-21', '2010-02-22', '2010-02-23',
               '2010-02-24', '2010-02-25', '2010-02-26', '2010-02-27',
               '2010-02-28', '2010-03-01',
               ...
               '2011-02-24', '2011-02-25', '2011-02-26', '2011-02-27',
               '2011-02-28', '2011-03-01', '2011-03-02', '2011-03-03',
               '2011-03-04', '2011-03-05'],
              dtype='datetime64[ns]', length=379, freq='D')

<a id='pandas-join'></a> [**Pandas - Merge, Join, Concatenate**](https://pandas.pydata.org/pandas-docs/stable/merging.html)

**Difference between '```append```, ```concat```, ```merge```, and ```join```**

**```append```**
* Solely for appending rows to a dataframe, but it is typically slow and seldomly used in favor of concat.
* Exists as a dataframe method (i.e. is called via df.append )

**```concat```**
* For stacking dataframes vertically or horizontally.
* Exists in the pandas namespace (i.e. is called via pd.concat)

**```merge```**
* For performing relational database style stitching

**```join```**
* A shortcut for merging on indices as opposed to merge which allows you to join along arbitrary columns.


<table border="1" class="colwidths-given docutils">
<colgroup>
<col width="20%">
<col width="20%">
<col width="60%">
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">Merge method</th>
<th class="head">SQL Join Name</th>
<th class="head">Description</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td><code class="docutils literal notranslate"><span class="pre">left</span></code></td>
<td><code class="docutils literal notranslate"><span class="pre">LEFT</span> <span class="pre">OUTER</span> <span class="pre">JOIN</span></code></td>
<td>Use keys from left frame only</td>
</tr>
<tr class="row-odd"><td><code class="docutils literal notranslate"><span class="pre">right</span></code></td>
<td><code class="docutils literal notranslate"><span class="pre">RIGHT</span> <span class="pre">OUTER</span> <span class="pre">JOIN</span></code></td>
<td>Use keys from right frame only</td>
</tr>
<tr class="row-even"><td><code class="docutils literal notranslate"><span class="pre">outer</span></code></td>
<td><code class="docutils literal notranslate"><span class="pre">FULL</span> <span class="pre">OUTER</span> <span class="pre">JOIN</span></code></td>
<td>Use union of keys from both frames</td>
</tr>
<tr class="row-odd"><td><code class="docutils literal notranslate"><span class="pre">inner</span></code></td>
<td><code class="docutils literal notranslate"><span class="pre">INNER</span> <span class="pre">JOIN</span></code></td>
<td>Use intersection of keys from both frames</td>
</tr>
</tbody>
</table>

In [29]:
# Define example dataframes and series
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                   index=[0, 1, 2, 3])

df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                   index=[4, 5, 6, 7])
 

df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                   index=[8, 9, 10, 11])
s1 = pd.Series(['X0', 'X1', 'X2', 'X3'], name='X')

s2 = pd.Series(['X0', 'X1', 'X2', 'X3'],
               index=['A', 'B', 'C', 'D'])

**Difference between '```append```, ```concat```, ```merge```, and ```join```**

**```append```**
* Solely for appending rows to a dataframe, but it is typically slow and seldomly used in favor of concat.
* Exists as a dataframe method (i.e. is called via df.append )

**```concat```**
* For stacking dataframes vertically or horizontally.
* Exists in the pandas namespace (i.e. is called via pd.concat)

**```merge```**
* For performing relational database style stitching

**```join```**
* A shortcut for merging on indices as opposed to merge which allows you to join along arbitrary columns.


#### Append

In [28]:
# Note that Series objects are more accurately thought of as row objects rather
# than column objects as they are typically displayed

result = df1.append(s2, ignore_index=True)
print('{}\n\n{}\n\n{} '.format(df1, s2, result))

    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1
2  A2  B2  C2  D2
3  A3  B3  C3  D3

A    X0
B    X1
C    X2
D    X3
dtype: object

    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1
2  A2  B2  C2  D2
3  A3  B3  C3  D3
4  X0  X1  X2  X3 


### Concat

In [33]:
################################################################################
# Notes
#
# - Argument must be a list of DataFrames or Series, or a dict
# - In this example all dataframes have the same columns and non-overlapping
################################################################################
result = pd.concat([df1, df2, df3])
print(result)

      A    B    C    D
0    A0   B0   C0   D0
1    A1   B1   C1   D1
2    A2   B2   C2   D2
3    A3   B3   C3   D3
4    A4   B4   C4   D4
5    A5   B5   C5   D5
6    A6   B6   C6   D6
7    A7   B7   C7   D7
8    A8   B8   C8   D8
9    A9   B9   C9   D9
10  A10  B10  C10  D10
11  A11  B11  C11  D11


In [39]:
################################################################################
# Notes
#
# - Can attach a distinguishing label for each constituent dataframe now we 
#     will have a multiindex
# - Concat makes a full copy of the data
################################################################################
result = pd.concat([df1, df2, df3], keys=['x', 'y', 'z'])
print(result)

        A    B    C    D
x 0    A0   B0   C0   D0
  1    A1   B1   C1   D1
  2    A2   B2   C2   D2
  3    A3   B3   C3   D3
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11


In [53]:
################################################################################
# Notes
#
# - The default merge method is 'outer' meaning that it will use the union of 
#     all keys in both dataframes
# - Note that with an outer join it fills all missing fields with nans
################################################################################
df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                   index=[2, 3, 6, 7])
# Concatenate along rows (i.e. vertical stacking)
result = pd.concat([df1, df4], axis=0)
print(result)
print('\n')
# Concatenate along columns (i.e. horizontal stacking)
result = pd.concat([df1, df4], axis=1)
print(result)

     A   B    C   D    F
0   A0  B0   C0  D0  NaN
1   A1  B1   C1  D1  NaN
2   A2  B2   C2  D2  NaN
3   A3  B3   C3  D3  NaN
2  NaN  B2  NaN  D2   F2
3  NaN  B3  NaN  D3   F3
6  NaN  B6  NaN  D6   F6
7  NaN  B7  NaN  D7   F7


     A    B    C    D    B    D    F
0   A0   B0   C0   D0  NaN  NaN  NaN
1   A1   B1   C1   D1  NaN  NaN  NaN
2   A2   B2   C2   D2   B2   D2   F2
3   A3   B3   C3   D3   B3   D3   F3
6  NaN  NaN  NaN  NaN   B6   D6   F6
7  NaN  NaN  NaN  NaN   B7   D7   F7


/home/casey/coding_interview/interview_venv/lib/python3.5/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [63]:
################################################################################
# Notes - Inner Joins
#
# - A join of inner only returns the columns and rows that can be filled in
#     completely, i.e. the intersection
################################################################################
df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                   index=[2, 3, 6, 7])
# Concatenate along rows (i.e. vertical stacking)
# Drops columns A, C since the are not present in df4, it drops F not in df1
result = pd.concat([df1, df4], join='inner', axis=0)
print(result)
print('\n')

# Concatenate along columns (i.e. horizontal stacking)
# Drops rows 0, 1, not in df4 and 6, 7 not in df1
result = pd.concat([df1, df4], join='inner', axis=1)
print(result)
print('\n')

# Concatenate along columns (i.e. horizontal stacking)
# We can perform something between an inner and an outer join by specifying we
# want to use only the keys from one but not the other (as opposed to an outer
# join which uses the union)
result = pd.concat([df1, df4], axis=1, join_axes=[df1.index])
print(result)
print('\n')

result = pd.concat([df4, df1], axis=1, join_axes=[df4.index])
print(result)
print('\n')

    B   D
0  B0  D0
1  B1  D1
2  B2  D2
3  B3  D3
2  B2  D2
3  B3  D3
6  B6  D6
7  B7  D7


    A   B   C   D   B   D   F
2  A2  B2  C2  D2  B2  D2  F2
3  A3  B3  C3  D3  B3  D3  F3


    A   B   C   D    B    D    F
0  A0  B0  C0  D0  NaN  NaN  NaN
1  A1  B1  C1  D1  NaN  NaN  NaN
2  A2  B2  C2  D2   B2   D2   F2
3  A3  B3  C3  D3   B3   D3   F3


    B   D   F    A    B    C    D
2  B2  D2  F2   A2   B2   C2   D2
3  B3  D3  F3   A3   B3   C3   D3
6  B6  D6  F6  NaN  NaN  NaN  NaN
7  B7  D7  F7  NaN  NaN  NaN  NaN




In [65]:
################################################################################
# Notes - resetting index
#
# - Can reset index and make sequential
################################################################################
df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                   index=[2, 3, 6, 7])
# Concatenate along rows (i.e. vertical stacking)
# Resets row numbers
result = pd.concat([df1, df4], axis=0, ignore_index=True)
print(result)
print('\n')

# Concatenate along columns (i.e. horizontal stacking)
# Resets column names
result = pd.concat([df1, df4], axis=1, ignore_index=True)
print(result)
print('\n')

     A   B    C   D    F
0   A0  B0   C0  D0  NaN
1   A1  B1   C1  D1  NaN
2   A2  B2   C2  D2  NaN
3   A3  B3   C3  D3  NaN
4  NaN  B2  NaN  D2   F2
5  NaN  B3  NaN  D3   F3
6  NaN  B6  NaN  D6   F6
7  NaN  B7  NaN  D7   F7


     0    1    2    3    4    5    6
0   A0   B0   C0   D0  NaN  NaN  NaN
1   A1   B1   C1   D1  NaN  NaN  NaN
2   A2   B2   C2   D2   B2   D2   F2
3   A3   B3   C3   D3   B3   D3   F3
6  NaN  NaN  NaN  NaN   B6   D6   F6
7  NaN  NaN  NaN  NaN   B7   D7   F7




/home/casey/coding_interview/interview_venv/lib/python3.5/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


### Merge

```python
pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)
```

In [72]:
################################################################################
# Notes - Merging on common column one-to-one
#
# - Think about merging as stitching or mappings
# - The default join is INNER
################################################################################
table1 = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                       'A': ['A0', 'A1', 'A2', 'A3'],
                       'B': ['B0', 'B1', 'B2', 'B3']})
 

table2 = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                       'C': ['C0', 'C1', 'C2', 'C3'],
                       'D': ['D0', 'D1', 'D2', 'D3']})

# Merging on common column  'key'
result = pd.merge(left=table1, right=table2, on='key')
print(result)
print('\n')

# Merging on common column  'key' in reverse order
# Since the keys are one-to-one these are the same (though the columns are permuted)
result = pd.merge(left=table2, right=table1, on='key')
print(result)
print('\n')

    A   B key   C   D
0  A0  B0  K0  C0  D0
1  A1  B1  K1  C1  D1
2  A2  B2  K2  C2  D2
3  A3  B3  K3  C3  D3


    C   D key   A   B
0  C0  D0  K0  A0  B0
1  C1  D1  K1  A1  B1
2  C2  D2  K2  A2  B2
3  C3  D3  K3  A3  B3




In [77]:
################################################################################
# Notes - Merging on multiple columns
#
# - The domain of the mapping is the left key, return all entries in right table
#     that match that key
# - If there are common columns in both tables which are not joined on then they
#     will be added but with suffixes '_x' for the left and '_y' for the right
################################################################################

table1 = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']}) 

table2 = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})
print(table1)
print('\n')
print(table2)
print('\n')

# Join on single key
result = pd.merge(left=table1, right=table2, on=['key1'])
print(result)
print('\n')

# Note that the join is commutative since we are performing an inner join
# The only difference is the suffixes added on the non-joined common column
# key2
result = pd.merge(left=table2, right=table1, on=['key1'])
print(result)
print('\n')

# Join on multiple keys means that both keys have to match both keys in
# the target table. So, since table2 has no entry with (key1, key2) = (K0, K1)
# this is not entered into the resulting (since this is an inner join)
result = pd.merge(left=table1, right=table2, on=['key1', 'key2'])
print(result)
print('\n')

result = pd.merge(left=table2, right=table1, on=['key1', 'key2'])
print(result)
print('\n')

    A   B key1 key2
0  A0  B0   K0   K0
1  A1  B1   K0   K1
2  A2  B2   K1   K0
3  A3  B3   K2   K1


    C   D key1 key2
0  C0  D0   K0   K0
1  C1  D1   K1   K0
2  C2  D2   K1   K0
3  C3  D3   K2   K0


    A   B key1 key2_x   C   D key2_y
0  A0  B0   K0     K0  C0  D0     K0
1  A1  B1   K0     K1  C0  D0     K0
2  A2  B2   K1     K0  C1  D1     K0
3  A2  B2   K1     K0  C2  D2     K0
4  A3  B3   K2     K1  C3  D3     K0


    C   D key1 key2_x   A   B key2_y
0  C0  D0   K0     K0  A0  B0     K0
1  C0  D0   K0     K0  A1  B1     K1
2  C1  D1   K1     K0  A2  B2     K0
3  C2  D2   K1     K0  A2  B2     K0
4  C3  D3   K2     K0  A3  B3     K1


    A   B key1 key2   C   D
0  A0  B0   K0   K0  C0  D0
1  A2  B2   K1   K0  C1  D1
2  A2  B2   K1   K0  C2  D2


    C   D key1 key2   A   B
0  C0  D0   K0   K0  A0  B0
1  C1  D1   K1   K0  A2  B2
2  C2  D2   K1   K0  A2  B2




In [87]:
################################################################################
# Notes - Full Outer Join
#
# - builds table with union of all keys from both left and right tables
################################################################################

table1 = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']}) 

table2 = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})

# Table will include all pairs from both left and right tables:
#  From left:  (K0, K0), (K0, K1), (K1, K0), (K2, K1)
#  From right: (K0, K0), (K1, K0), (K1, K0), (K2, K0)
# All elements which do not have a counterpart will be filled with NaNs
result = pd.merge(left=table1, right=table2, how='outer', on=['key1', 'key2'])
print(result)
print('\n')

# Note that up to column ordering the two tables are the same
result = pd.merge(left=table2, right=table1, how='outer', on=['key1', 'key2'])
print(result)
print('\n')

result = pd.merge(left=table1, right=table2, how='outer', on=['key1'])
print(result)
print('\n')

# Note that up to column ordering the two tables are the same
result = pd.merge(left=table2, right=table1, how='outer', on=['key1'])
print(result)
print('\n')

     A    B key1 key2    C    D
0   A0   B0   K0   K0   C0   D0
1   A1   B1   K0   K1  NaN  NaN
2   A2   B2   K1   K0   C1   D1
3   A2   B2   K1   K0   C2   D2
4   A3   B3   K2   K1  NaN  NaN
5  NaN  NaN   K2   K0   C3   D3


     C    D key1 key2    A    B
0   C0   D0   K0   K0   A0   B0
1   C1   D1   K1   K0   A2   B2
2   C2   D2   K1   K0   A2   B2
3   C3   D3   K2   K0  NaN  NaN
4  NaN  NaN   K0   K1   A1   B1
5  NaN  NaN   K2   K1   A3   B3


    A   B key1 key2_x   C   D key2_y
0  A0  B0   K0     K0  C0  D0     K0
1  A1  B1   K0     K1  C0  D0     K0
2  A2  B2   K1     K0  C1  D1     K0
3  A2  B2   K1     K0  C2  D2     K0
4  A3  B3   K2     K1  C3  D3     K0


    C   D key1 key2_x   A   B key2_y
0  C0  D0   K0     K0  A0  B0     K0
1  C0  D0   K0     K0  A1  B1     K1
2  C1  D1   K1     K0  A2  B2     K0
3  C2  D2   K1     K0  A2  B2     K0
4  C3  D3   K2     K0  A3  B3     K1




In [93]:
################################################################################
# Notes - Left/Right Outer Join
#
# - Builds table using all of the keys in either the left or right table
################################################################################

table1 = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']}) 

table2 = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})

# This will have all of the keys from the left table. If there is no corresponding
# element in the right table, they will be filled with NaNs. Also, a left key can
# be duplicated if there are multiple entries in the right table with the same key
result = pd.merge(left=table1, right=table2, how='left', on=['key1', 'key2'])
print(result)
print('\n')

# Note that this is NON-COMMUTATIVE
result = pd.merge(left=table2, right=table1, how='left', on=['key1', 'key2'])
print(result)
print('\n')

# This will have all of the keys in the right table
result = pd.merge(left=table1, right=table2, how='right', on=['key1', 'key2'])
print(result)
print('\n')

# Note however that left<->right and table1<->table2 is commutative
result = pd.merge(left=table2, right=table1, how='right', on=['key1', 'key2'])
print(result)
print('\n')

    A   B key1 key2    C    D
0  A0  B0   K0   K0   C0   D0
1  A1  B1   K0   K1  NaN  NaN
2  A2  B2   K1   K0   C1   D1
3  A2  B2   K1   K0   C2   D2
4  A3  B3   K2   K1  NaN  NaN


    C   D key1 key2    A    B
0  C0  D0   K0   K0   A0   B0
1  C1  D1   K1   K0   A2   B2
2  C2  D2   K1   K0   A2   B2
3  C3  D3   K2   K0  NaN  NaN


     A    B key1 key2   C   D
0   A0   B0   K0   K0  C0  D0
1   A2   B2   K1   K0  C1  D1
2   A2   B2   K1   K0  C2  D2
3  NaN  NaN   K2   K0  C3  D3


     C    D key1 key2   A   B
0   C0   D0   K0   K0  A0  B0
1   C1   D1   K1   K0  A2  B2
2   C2   D2   K1   K0  A2  B2
3  NaN  NaN   K0   K1  A1  B1
4  NaN  NaN   K2   K1  A3  B3




### Join

Join is really just a shorthand for merging on the indexes of the two tables by default. However, you can include a column argument to join to match a column in the left to an index on the right.
i.e. the two are the same

```python
left.join(right, on=key_or_keys)
pd.merge(left=left, right=right, left_on=key_or_keys, right_index=True, how='left')
```

In [119]:
################################################################################
# Notes - .join()
#
# - Shorthand designed for joining on indexes
# - Default is a left outer join
################################################################################

table1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                       'B': ['B0', 'B1', 'B2', 'B3'],
                       'key': ['K0', 'K1', 'K0', 'K1']})

table2 = pd.DataFrame({'C': ['C0', 'C1'],
                       'D': ['D0', 'D1']},
                       index=['K0', 'K1'])

table3 = pd.DataFrame({'A': ['E0', 'E1', 'E2'],
                       'B': ['D0', 'D1', 'D2'],
                       'key': ['K0', 'K1', 'K1',]})

table4 = pd.DataFrame({'F': ['F0', 'F1', 'F2'],
                       'G': ['G0', 'G1', 'G2'],
                       'key': ['K0', 'K1', 'K1',]})

# Note that since table 2's index is not a set of integers, but a set of strings
# So, when it attempts to join, it will try to join on their indexes, but it
# can't match any so all of right columns are filled with NaNs
result = table1.join(table2)
print(result)
print('\n')

# Now we tell it to instead join on table1's column='key' which finds matches in
# table2's index
result = table1.join(table2, on='key')
print(result)
print('\n')

# Here will be able to match their indexes since they are both just row numbers.
# However, the tables have columns with the same name, so we need to specify what
# suffixes we want to add to the left or right column names to distinguish them
result = table1.join(table3, lsuffix='_l')
print(result)
print('\n')

result = table1.join(table4, lsuffix='_l', rsuffix='_r')
print(result)
print('\n')

    A   B key    C    D
0  A0  B0  K0  NaN  NaN
1  A1  B1  K1  NaN  NaN
2  A2  B2  K0  NaN  NaN
3  A3  B3  K1  NaN  NaN


    A   B key   C   D
0  A0  B0  K0  C0  D0
1  A1  B1  K1  C1  D1
2  A2  B2  K0  C0  D0
3  A3  B3  K1  C1  D1


  A_l B_l key_l    A    B  key
0  A0  B0    K0   E0   D0   K0
1  A1  B1    K1   E1   D1   K1
2  A2  B2    K0   E2   D2   K1
3  A3  B3    K1  NaN  NaN  NaN


    A   B key_l    F    G key_r
0  A0  B0    K0   F0   G0    K0
1  A1  B1    K1   F1   G1    K1
2  A2  B2    K0   F2   G2    K1
3  A3  B3    K1  NaN  NaN   NaN




**Difference between '```append```, ```concat```, ```merge```, and ```join```**

**```append```**
* Solely for appending rows to a dataframe, but it is typically slow and seldomly used in favor of concat.
* Exists as a dataframe method (i.e. is called via df.append )

**```concat```**
* For stacking dataframes vertically or horizontally.
* Exists in the pandas namespace (i.e. is called via pd.concat)

**```merge```**
* For performing relational database style stitching

**```join```**
* A shortcut for merging on indices as opposed to merge which allows you to join along arbitrary columns.


In [6]:
import pandas as pd
import numpy as np

# Functions used to generate mock data simulating the tables given above.

n_students = 1000
n_days = 10
start_date = '2017-09-01'
end_date = '2018-06-15'

################################################################################
# Attendance Table                                                             #
################################################################################
def _make_event_dates(n_students, start_date, end_date):
    dr = pd.date_range(start_date, end_date)
    dates = []
    for day in dr:
        dates.extend([day] * n_students)
    return dates

def _make_student_ids(n_students, n_days=None):
    student_ids = [xx for xx in range(100, 100 + n_students)]
    if n_days is not None:
        student_ids = student_ids * n_days
    return student_ids

def _make_attendance(n_students, n_days):
    return list(np.random.choice(2, n_students, p=[0.3, 0.7])) * n_days

def build_attendance_events(n_students, start_date, end_date):
    columns = ['date', 'student_id', 'attendance']
    n_days = len(pd.date_range(start_date, end_date))
    
    dates       = _make_event_dates(n_students, start_date, end_date)
    student_ids = _make_student_ids(n_students, n_days)
    attendance  = _make_attendance(n_students, n_days)
    data = [xx for xx in zip(dates, student_ids, attendance)]
    
    df = pd.DataFrame(data=data, columns=columns)
    return df

################################################################################
# District All Students Table                                                  #
################################################################################
def _make_school_ids(n_students):
    schools = ['South River High School',
               'New Brunswick High School',
               'East Brunswick High School',
               'Edison High School']
    return list(np.random.choice(schools, n_students))

def _make_grade_levels(n_students):
    grades = ['Freshman', 'Sophomore', 'Junior', 'Senior']
    return list(np.random.choice(grades, n_students))
    
def _make_DOBs(grade_levels):
    birth_years = {
        'Freshman': 2005,
        'Sophomore': 2004,
        'Junior': 2003,
        'Senior': 2002
    }
    years = [birth_years[xx] for xx in grade_levels]
    months = list(np.random.choice(np.arange(1,13), len(grade_levels)))
    days = list(np.random.choice(np.arange(1,29), len(grade_levels)))
    DOBs = pd.to_datetime(['{}-{}-{}'.format(*dd) for dd in zip(months, days, years)])
    return DOBs

def _make_hometowns(school_ids):
    hometowns = [school.split('High School')[0].strip() for school in school_ids]
    return hometowns
    

def build_all_students(n_students):
    student_ids  = _make_student_ids(n_students)
    school_ids   = _make_school_ids(n_students)
    grade_levels = _make_grade_levels(n_students)
    DOBs         = _make_DOBs(grade_levels)
    hometowns    = _make_hometowns(school_ids)
    
    columns = ['student_id', 'school_id', 'grade_level', 'date_of_birth', 'hometown']
    data = [xx for xx in zip(student_ids, school_ids, grade_levels, DOBs, hometowns)]
    df = pd.DataFrame(data=data, columns=columns)
    return df
    

attendance_events = build_attendance_events(n_students, start_date, end_date)
all_students = build_all_students(n_students=n_students)

<a id='pandas-groupby'></a> [**Pandas - Group By: split-apply-combine**](https://pandas.pydata.org/pandas-docs/stable/groupby.html)